# import


In [1]:
import sys
import warnings
warnings.filterwarnings('ignore')


import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.3f}'.format
import matplotlib.pyplot as plt
import koreanize_matplotlib
%config InlineBackend.figure_format = 'retina'
import seaborn as sns

root_path = f".."
data_path = f"{root_path}/data"
data_olist_path = f"{data_path}/olist"

In [2]:
def get_var_name(variable):
    globals_dict = globals()

    return [var_name for var_name in globals_dict if globals_dict[var_name] is variable]

class EDA:

    def __init__(
            self:pd.DataFrame,
            df,
            idx=None,
            x=None,
            y=None,
            fig_col_len=2,
            ) -> pd.DataFrame:

        self.idx = idx
        if idx != None:
            temp = df.set_index(idx)
            self.df = temp
        else:
            self.df = df

        if x == None:
            self.x = df.columns.to_list()
        else:
            self.x = x
            
        self.y = y
        self.fig_col_len = fig_col_len
        
    def print_title(body, br=2, bp="┌▣ ", hr=" ---- ---- ---- ----"):

        """
        body : 내용
        bp : bullet point, 글머리 기호
        hr : Horizontal Rule, 수평선
        """

        class ff:
            PURPLE = "\033[95m"
            CYAN = "\033[96m"
            DARKCYAN = "\033[36m"
            BLUE = "\033[94m"
            GREEN = "\033[92m"
            YELLOW = "\033[93m"
            RED = "\033[91m"
            BOLD = "\033[1m"
            UNDERLINE = "\033[4m"
            END = "\033[0m"

        print("\n" * br + ff.BOLD + bp + ff.UNDERLINE + body + ff.END + hr)
        

    def Check_df(df):

        EDA.print_title("""df.shape""")
        print(df.shape)


        EDA.print_title("""df.info()""")
        print(df.info())

        EDA.print_title("""df.head()""")
        display(df.head())

        EDA.print_title("""df.columns.to_list()""")
        print(df.columns.to_list())
        
    def Check(self) -> pd.DataFrame:

        df = self.df
        EDA.Check_df(df)    
        
    def Summary_df(df):
        
        EDA.print_title("""df.describe().T""")
        display(df.describe().T)

        try:
            EDA.print_title("""df.describe(include=['O'])""")
            display(df.describe(include=["O"]))
        except:
            pass

        # 결측치
        df_temp = df.isna()
        EDA.print_title("""df.isna().sum()""")
        display(df_temp.sum().to_frame())

        EDA.print_title("""sns.heatmap(data=df.isna())""")
        sns.heatmap(data=df.isna(), cmap='Greys')
        plt.title("""sns.heatmap(data=df.isna())""", fontsize=20)
        plt.show()

        EDA.print_title("df.isna().mean()")
        display(df_temp.mean().to_frame())

    def Summary(self):
        
        df = self.df
        EDA.Summary_df(df)

    def mv(self) -> None:

        df = self.df

        fig_col_len = self.fig_col_len
        import math
        fig_row_len = math.ceil(len(self.x) / fig_col_len)
        
        sns.pairplot(data=df, height=len(self.x)*4)
        

        df_corr = df.corr()
        EDA.print_title("""df.corr()""")
        display(df_corr)
        EDA.print_title("""sns.heatmap(data=df.corr(), annot=True, fmt=".2f")""")
        plt.figure(figsize=(20,7))
        sns.heatmap(data=df_corr, annot=True, fmt=".2f", annot_kws={"size": 14}, cmap="Blues")
        plt.show()
        

# 재구매율 Repurchase Rate

- 재구매율 = 재구매 / (처음 구매 + 재구매) * 100

- 1달 내 재구매율
  
        https://byline.network/2022/10/1004_02/

## Data Load

In [3]:
df_raw = pd.read_csv(f"{data_olist_path}/complete_orders.csv", index_col=0)

### customer_unique_id의 freq확인 (1이면 안됨)

In [4]:
df_raw.describe(include="O")

customer_id order_status  \
count                              92657        92657   
unique                             92657            2   
top     9ef432eb6251297304e76186b10a928d    delivered   
freq                                   1        92651   

       order_purchase_timestamp    order_approved_at  \
count                     92657                92657   
unique                    92172                85209   
top         2018-08-02 12:05:26  2018-02-27 04:31:10   
freq                          3                    9   

       order_delivered_carrier_date order_delivered_customer_date  \
count                         92657                         92657   
unique                        77187                         91897   
top             2018-05-09 15:48:00           2016-10-27 17:32:07   
freq                             47                             3   

       order_estimated_delivery_date                customer_unique_id  \
count                          92657                             92657   
unique                           444                             89732   
top              2017-12-20 00:00:00  8d50f5eadf50201ccdcedfb9e2ac8455   
freq                             497                                14   

       customer_city customer_state                         seller_id  \
count          92657          92657                             92657   
unique          4005             27                              2881   
top        sao paulo             SP  6560211a19b47992c3666cc44a7e94c0   
freq           14459          39052                              1730   

       seller_city seller_state                        product_id  \
count        92657        92657                             92657   
unique         586           22                             30345   
top      sao paulo           SP  aca2eb7d00ea1a7b8ebd4e68314663af   
freq         22979        65703                               413   

        shipping_limit_date payment_type product_category_name_english  
count                 92657        92657                         92657  
unique                87714            4                            71  
top     2018-06-11 03:31:04  credit_card                bed_bath_table  
freq                      6        70200                          8908

## 파생 컬럼 만들기

1. customer_unique_id의 최초 구매일
2. order_id의 구매일과 customer_unique_id의의 최초 구매일의 차이
3. order_id의 구매일이 customer_unique_id의 N번 째
4. order_id의 구매일 customer_unique_id의 이전 order_id의 구매일의 차이

### 첫구매일 (order_purchase_timestamp_First)

In [6]:
df = df_raw.copy()

firstorder = (
    df.groupby("customer_unique_id")["order_purchase_timestamp"]
    .min()
    .to_frame()
    .reset_index()
)
firstorder.columns = ["customer_unique_id", "order_purchase_timestamp_First"]

df = pd.merge(df, firstorder, on="customer_unique_id")
df.describe(include=['O'])

order_id                       customer_id  \
count                              92657                             92657   
unique                             92657                             92657   
top     e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
freq                                   1                                 1   

       order_status                customer_unique_id customer_city  \
count         92657                             92657         92657   
unique            2                             89732          4005   
top       delivered  8d50f5eadf50201ccdcedfb9e2ac8455     sao paulo   
freq          92651                                14         14459   

       customer_state                         seller_id seller_city  \
count           92657                             92657       92657   
unique             27                              2881         586   
top                SP  6560211a19b47992c3666cc44a7e94c0   sao paulo   
freq            39052                              1730       22979   

       seller_state                        product_id  shipping_limit_date  \
count         92657                             92657                92657   
unique           22                             30345                87714   
top              SP  aca2eb7d00ea1a7b8ebd4e68314663af  2018-06-11 03:31:04   
freq          65703                               413                    6   

       payment_type product_category_name_english  
count         92657                         92657  
unique            4                            71  
top     credit_card                bed_bath_table  
freq          70200                          8908

### N번째 구매 ()

In [7]:
df['ordertimerank'] = (
    df.groupby("customer_unique_id")["order_purchase_timestamp"]
    .rank()
)
df.describe()

zip_code_prefix_customer  geolocation_lat_customer  \
count                 92657.000                 92657.000   
mean                  35105.345                   -21.205   
std                   29822.372                     5.604   
min                    1004.000                   -33.692   
25%                   11320.000                   -23.591   
50%                   24358.000                   -22.926   
75%                   58463.000                   -20.137   
max                   99980.000                    42.184   

       geolocation_lng_customer  zip_code_prefix_seller  \
count                 92657.000               92657.000   
mean                    -46.190               24730.654   
std                       4.062               27790.147   
min                     -72.664                1001.000   
25%                     -48.113                6429.000   
50%                     -46.632               13660.000   
75%                     -43.632               29156.000   
max                      -4.948               99730.000   

       geolocation_lat_seller  geolocation_lng_seller  order_item_id  \
count               92657.000               92657.000      92657.000   
mean                  -22.795                 -47.236          1.125   
std                     2.754                   2.358          0.508   
min                   -32.096                 -63.896          1.000   
25%                   -23.616                 -48.810          1.000   
50%                   -23.419                 -46.750          1.000   
75%                   -21.755                 -46.521          1.000   
max                    -2.506                 -34.854         21.000   

          price  freight_value  review_score  payment_installments  \
count 92657.000      92657.000     92657.000             92657.000   
mean    125.384         20.156         4.155                 2.915   
std     188.983         15.785         1.285                 2.708   
min       0.850          0.000         1.000                 0.000   
25%      41.300         13.270         4.000                 1.000   
50%      79.000         16.360         5.000                 2.000   
75%     139.900         21.220         5.000                 4.000   
max    6735.000        409.680         5.000                24.000   

               delivered_purchase          estimated_purchase  \
count                       92657                       92657   
mean   12 days 13:42:31.202348446  23 days 17:52:14.614243932   
std     9 days 12:45:16.689113917   8 days 18:52:55.477069463   
min               0 days 12:48:07             2 days 00:11:32   
25%               6 days 19:04:17            18 days 07:59:42   
50%              10 days 05:54:42            23 days 05:34:10   
75%              15 days 17:53:21            28 days 10:05:51   
max             239 days 12:56:59           155 days 03:15:04   

                delivered_estimated  ordertimerank  
count                         92657      92657.000  
mean   -12 days +19:50:16.588104515          1.037  
std      10 days 04:16:02.590293587          0.235  
min             -147 days +23:36:47          1.000  
25%              -17 days +18:18:37          1.000  
50%              -12 days +03:33:07          1.000  
75%               -7 days +14:52:23          1.000  
max               215 days 07:58:32         14.000

### 구매일 - 첫구매일 (order_purchase_timestamp - order_purchase_timestamp_First)

In [8]:
df["purchase_firstorder"] = df["order_purchase_timestamp"] - df["order_purchase_timestamp_First"]

### 이전 구매일과의 차이

### 재구매율 구하기